In [2]:
import docx2txt

temp = docx2txt.process("Amar_Resume.docx")
text = [line.replace('\t',' ') for line in temp.split('\n') if line]
text=' '.join(text)

In [3]:
import spacy
from spacy.matcher import Matcher
import re
import docx2txt

temp = docx2txt.process("Amar_Resume.docx")
text = [line.replace('\t',' ') for line in temp.split('\n') if line]
text=' '.join(text)

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_details(text):
    nlp_text = nlp(text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', None, pattern)
    
    matches = matcher(nlp_text)
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        print("Name:",span.text)
        break
    phone = re.search(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]',text)
    print("mobile:",phone[0])
    
    address= re.search('([A-Z]{1}[a-z]+) ([0-9]){6}',text)
    print("Address : ",address[0])
    import pandas as pd
    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("skills.csv") 
    
    # extract values
    skills = list(data.columns.values)
    
    skillset = []
    
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    print("SKills :",[i.capitalize() for i in set([i.lower() for i in skillset])])
    
    PAN= re.search('(?m)([a-zA-Z])+([0-9])+.',text)
    print("PAN NO:",PAN[0])
    

In [4]:
extract_details(text)

Name: Amar Latchireddy
mobile: +919515152743
Address :  Banglore 560027
SKills : ['Python', 'Sqoop', 'Ml', 'Sql', 'Hbase', 'Css', 'Ecommerce', 'Content', 'Java', 'Html', 'Hadoop', 'Ai', 'Nlp', 'Algorithms', 'Programming', 'Pig']
PAN NO: ALVPL9089A


In [5]:
import re

def extract_mobile_number(text):
    phone = re.findall(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]',text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return  number
        else:
            return number

In [6]:
extract_mobile_number(text)

'+919515152743'

In [7]:
import re

def extract_email(email):
    email = re.findall(",([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [8]:
extract_email(text)

'amarlatchireddy@gmail.com'

In [9]:
import pandas as pd
import spacy

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("skills.csv") 
    
    # extract values
    skills = list(data.columns.values)
    
    skillset = []
    
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

In [10]:
extract_skills(text)

['Python',
 'Sqoop',
 'Ml',
 'Sql',
 'Hbase',
 'Css',
 'Ecommerce',
 'Content',
 'Java',
 'Html',
 'Hadoop',
 'Ai',
 'Nlp',
 'Algorithms',
 'Programming',
 'Pig']

In [11]:
import re
import spacy
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII','M.C.A','MCA','INTERMEDIATE','BSC','MATRICULATION','B.SC','B.S.C','BSC-(MPCS)','B.Sc(MPCS)','PH.D','MASTER OF COMPUTER APPILICATIONS'
        ]

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,|-]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

In [12]:
text


'Amar Latchireddy EDUCATION M.C.A  SRI VENKATESWARA UNIVERSITY  –TIRUPATI Year of passing: 2019 | CGPA: 7.9 EDUCATION M.C.A  SRI VENKATESWARA UNIVERSITY  –TIRUPATI Year of passing: 2019 | CGPA: 7.9 IMDB REVIEW SENTIMENT CLASIFICATION using LSTM SEQUENCE CLASSIFICATION OF REVIEW USING LSTM Applied Embedding layer and Dropout layer. Trained using LSTM model | Accuracy achieved 86.54 Skills: Sequence Classification, NLP, LSTM IMDB REVIEW SENTIMENT CLASIFICATION using LSTM SEQUENCE CLASSIFICATION OF REVIEW USING LSTM Applied Embedding layer and Dropout layer. Trained using LSTM model | Accuracy achieved 86.54 Skills: Sequence Classification, NLP, LSTM I am seeking a machine learning role with a company that is both challenging and reward, where I can use my skills and knowledge to build state of the art models using cutting edge technologies. I am seeking a machine learning role with a company that is both challenging and reward, where I can use my skills and knowledge to build state of th

In [13]:
extract_education(text)

[('MCA', '2019'),
 ('BSC', '2016'),
 ('INTERMEDIATE', '2013'),
 ('MATRICULATION', '2011')]

In [14]:
text

'Amar Latchireddy EDUCATION M.C.A  SRI VENKATESWARA UNIVERSITY  –TIRUPATI Year of passing: 2019 | CGPA: 7.9 EDUCATION M.C.A  SRI VENKATESWARA UNIVERSITY  –TIRUPATI Year of passing: 2019 | CGPA: 7.9 IMDB REVIEW SENTIMENT CLASIFICATION using LSTM SEQUENCE CLASSIFICATION OF REVIEW USING LSTM Applied Embedding layer and Dropout layer. Trained using LSTM model | Accuracy achieved 86.54 Skills: Sequence Classification, NLP, LSTM IMDB REVIEW SENTIMENT CLASIFICATION using LSTM SEQUENCE CLASSIFICATION OF REVIEW USING LSTM Applied Embedding layer and Dropout layer. Trained using LSTM model | Accuracy achieved 86.54 Skills: Sequence Classification, NLP, LSTM I am seeking a machine learning role with a company that is both challenging and reward, where I can use my skills and knowledge to build state of the art models using cutting edge technologies. I am seeking a machine learning role with a company that is both challenging and reward, where I can use my skills and knowledge to build state of th

In [15]:
PAN= re.search('(?m)([a-zA-Z])+([0-9])+.',text)
print("PAN NO:",PAN[0])

PAN NO: ALVPL9089A


In [16]:
address= re.search('([A-Z]{1}[a-z]+) ([0-9]){6}',text)
print("Address : ",address[0])

Address :  Banglore 560027


In [17]:
def extract_details(text):
    nlp_text = nlp(text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', None, pattern)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return (print ("Name:",span.text))
    

In [18]:
extract_details(text)

Name: Amar Latchireddy
